# Box Office Data Scraping and Analysis 
In this notebook, we'll scrape and analyze box office data from The-Numbers.com for the years 2020-2025. We'll collect daily gross information, extract movie details, and create a comprehensive dataset for analysis.

Let's begin by installing the necessary packages.


In [ ]:
!pip install cloudscraper

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 5.4 MB/s eta 0:00:00


## Importing Libraries

In this section, we'll import all the necessary libraries for our web scraping project. We use BeautifulSoup for parsing HTML content, pandas for data manipulation, and various other tools for handling dates, progress tracking, and concurrent processing.

CloudScraper is particularly important because The-Numbers.com employs anti-bot measures that regular requests libraries might not handle well. By using CloudScraper, we can bypass these protections and access the data we need reliably.

## Setting Up Web Scraping Tools

Web scraping requires careful handling of HTTP requests to avoid being blocked or rate-limited. In this section, we set up robust tools to extract movie data from The-Numbers.com.

First, we'll create a function that fetches web content and converts it to a BeautifulSoup object for easy parsing. This function will handle the initial retrieval of HTML data from any URL we provide.

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import cloudscraper

from bs4 import BeautifulSoup


def extract_movies(url, session=None):
    """
    Fetches and returns a BeautifulSoup object for the given URL,
    using a CloudScraper session configured with retries.
    """
    if session is None:
        # Fallback to a default session if none provided
        session = get_cloudscraper_session()

    response = session.get(url)
    # Raises an HTTPError if the status is 4xx or 5xx.
    response.raise_for_status()

    return BeautifulSoup(response.text, "html.parser")


## Creating a Robust Scraping Session

Websites often implement various protections against automated scraping. To overcome these challenges, we need to create a specialized session that can:

1. Bypass Cloudflare protection
2. Automatically retry failed requests
3. Use appropriate delays between requests
4. Present itself with a legitimate user agent

The following function creates such a session with retry capabilities for different HTTP error codes.

In [ ]:
import cloudscraper
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry


def get_cloudscraper_session(
    total_retries=5,
    backoff_factor=1,
    status_forcelist=(403, 429, 500, 502, 503, 504),
    user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
    "(KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36",
):
    """
    Returns a CloudScraper session configured to retry on specific HTTP errors.
    """

    # Create a CloudScraper session (handles Cloudflare anti-bot challenges).
    scraper = cloudscraper.create_scraper()

    # Optionally update the User-Agent header.
    scraper.headers.update({"User-Agent": user_agent})

    # Configure the Retry strategy.
    retries = Retry(
        total=total_retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retries)

    # Mount the adapter to handle both HTTP and HTTPS.
    scraper.mount("http://", adapter)
    scraper.mount("https://", adapter)

    return scraper


## Alternative Session Configuration

We're implementing an alternative session creation function with similar parameters. Having multiple approaches gives us flexibility if we encounter different types of website protections or if one method becomes ineffective.

Both session functions use exponential backoff strategies, which means they wait progressively longer between retries, reducing the likelihood of being blocked for making too many requests too quickly.

In [ ]:
import cloudscraper
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry


def get_session(
    total_retries=5,
    backoff_factor=1,
    status_forcelist=(403, 429, 500, 502, 503, 504),
    user_agent=(
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
    ),
):
    """
    Returns a CloudScraper session object with built-in retry logic.
    """
    scraper = cloudscraper.create_scraper()
    scraper.headers.update({"User-Agent": user_agent})

    retries = Retry(
        total=total_retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retries)

    # Mount the adapter for HTTP/HTTPS
    scraper.mount("http://", adapter)
    scraper.mount("https://", adapter)

    return scraper


## Parsing Box Office Data

Now we'll implement a function to extract structured data from the daily box office charts on The-Numbers.com. This function handles the complex task of:

- Finding the right table in the HTML
- Extracting each movie's information
- Handling truncated titles and distributor names
- Organizing all data into a clean DataFrame

The function includes special logic to reconstruct full movie titles when they appear truncated (with "…") on the website by using information from the URL.

In [ ]:
import re
from bs4 import BeautifulSoup
import pandas as pd


def parse_html_to_dataframe(soup, date):
    data = []

    # Find the table in the HTML
    table = soup.find("table", {"id": "box_office_daily_table"})
    if not table:
        return pd.DataFrame()  # No table found, return empty

    rows = table.find_all("tr")

    for row in rows[1:-2]:  # Skip header and footer rows
        cols = row.find_all("td")
        if len(cols) >= 11:
            movie_link_tag = cols[2].find("a")
            movie_title = movie_link_tag.text.strip() if movie_link_tag else ""

            href_movie = movie_link_tag.get("href", "")
            # If movie title is truncated ("…"), reconstruct from href
            if "…" in movie_title and movie_link_tag:
                if "/movie/" in href_movie:
                    # Remove leading "/movie/"
                    title_from_href = re.sub(r"^/movie/", "", href_movie)
                    # Remove trailing "#..." (e.g., "#tab=box-office")
                    title_from_href = re.sub(r"#.*$", "", title_from_href)
                    # Remove any parentheses and their contents
                    title_from_href = re.sub(r"\(.*?\)", "", title_from_href)
                    # Replace hyphens with spaces
                    title_from_href = title_from_href.replace("-", " ")
                    # Clean up extra spaces
                    title_from_href = re.sub(r"\s+", " ", title_from_href).strip()
                    movie_title = title_from_href

            distributor_link_tag = cols[3].find("a")
            distributor = (
                distributor_link_tag.text.strip()
                if distributor_link_tag
                else cols[3].text.strip()
            )

            # If distributor is truncated ("…"), reconstruct from href
            if "…" in distributor and distributor_link_tag:
                href = distributor_link_tag.get("href", "")
                if "/distributor/" in href or "/company/" in href or "/studio/" in href:
                    # 1) Remove any leading path up to 'distributor/' (or 'company/', etc.)
                    dist_from_href = re.sub(
                        r".*/(distributor|company|studio)/", "", href
                    )
                    # 2) Remove trailing "#..." (if present)
                    dist_from_href = re.sub(r"#.*$", "", dist_from_href)
                    # 3) Remove parentheses if any
                    dist_from_href = re.sub(r"\(.*?\)", "", dist_from_href)
                    # 4) Replace hyphens with spaces
                    dist_from_href = dist_from_href.replace("-", " ")
                    # 5) Clean up extra spaces
                    dist_from_href = re.sub(r"\s+", " ", dist_from_href).strip()

                    distributor = dist_from_href

            gross = cols[4].text.strip()
            o_oyd = cols[5].text.strip()
            o_olw = cols[6].text.strip()
            theaters = cols[7].text.strip()
            per_theater = cols[8].text.strip()
            total_gross = cols[9].text.strip()
            days_in_release = cols[10].text.strip()

            data.append(
                [
                    movie_title,
                    distributor,
                    gross,
                    o_oyd,
                    o_olw,
                    theaters,
                    per_theater,
                    total_gross,
                    days_in_release,
                    date,
                    href_movie,
                ]
            )

    df = pd.DataFrame(
        data,
        columns=[
            "Movie Title",
            "Distributor",
            "Gross",
            "%YD",
            "%LW",
            "Theaters",
            "Per Theater",
            "Total Gross",
            "Days In Release",
            "Date",
            "link",
        ],
    )

    return df


## Sequential Data Collection

Our first data collection approach processes one day at a time in sequence. While this method is slower than parallel processing, it's easier to debug and places less strain on the website's servers.

This function will:
1. Loop through each date in our target range
2. Fetch the box office data for that day
3. Parse it into a structured format
4. Combine all daily data into one comprehensive DataFrame

We use tqdm to display a progress bar so we can track how the collection is proceeding.

In [ ]:
def collect_data_over_dates(start_date, end_date):
    current_date = datetime.strptime(start_date, "%Y/%m/%d")
    end_date = datetime.strptime(end_date, "%Y/%m/%d")

    all_data = pd.DataFrame()
    total_days = (end_date - current_date).days + 1
    with tqdm(total=total_days) as pbar:
        while current_date <= end_date:
            soup = extract_movies(
                f"https://www.the-numbers.com/box-office-chart/daily/{current_date.strftime('%Y')}/{current_date.strftime('%m')}/{current_date.strftime('%d')}#box_office_daily_table=od4"
            )
            daily_df = parse_html_to_dataframe(soup, current_date.strftime("%Y/%m/%d"))
            all_data = pd.concat([all_data, daily_df], ignore_index=True)
            current_date += timedelta(days=1)
            pbar.update(1)

    return all_data

## Error Tracking

Web scraping at scale often encounters occasional failures due to network issues, website changes, or temporary blocks. To ensure we capture these issues, we'll create a list to track any URLs that fail during our data collection process. This helps us identify problematic dates that might need special handling.

In [ ]:
errors = []


## Parallel Data Collection

To dramatically speed up our data collection, we'll implement a parallel processing approach using Python's ThreadPoolExecutor. This allows us to fetch data for multiple dates simultaneously, potentially reducing collection time by a factor of 10-100x.

Each thread handles a single date, fetching and parsing the data independently. We then combine all results into a single DataFrame. The error handling ensures that if any individual request fails, it won't crash the entire process.

In [ ]:
def collect_data_over_dates_parallel(start_date, end_date, extract_movies):
    def fetch_and_parse(current_date):
        try:
            url = f"https://www.the-numbers.com/box-office-chart/daily/{current_date.strftime('%Y/%m/%d')}#box_office_daily_table=od4"
            html_content = extract_movies(url)
            return parse_html_to_dataframe(
                html_content, current_date.strftime("%Y/%m/%d")
            )
        except Exception as e:
            errors.append(url)
            return pd.DataFrame()

    start = datetime.strptime(start_date, "%Y/%m/%d")
    end = datetime.strptime(end_date, "%Y/%m/%d")
    dates = [start + timedelta(days=x) for x in range((end - start).days + 1)]

    all_data = pd.DataFrame()

    with ThreadPoolExecutor(max_workers=100) as executor:
        results = list(tqdm(executor.map(fetch_and_parse, dates), total=len(dates)))

    all_data = pd.concat(results, ignore_index=True)

    return all_data

## Executing the Data Collection

Now we'll run our parallel data collection function to gather box office data from January 1, 2020 through March 1, 2025. 

This collection process may take some time as we're gathering data for over 1,800 days, but our parallel approach makes it much faster than sequential processing.

In [ ]:
df = collect_data_over_dates_parallel("2020/01/01", "2025/03/01", extract_movies)

100%|██████████| 1887/1887 [02:38<00:00, 11.92it/s]


## Creating a DataFrame of Unique Movies

In our daily box office data, the same movies appear multiple times across different dates. To efficiently collect additional information, we'll create a new DataFrame containing only unique movies and their links.

First, we extract just the movie titles and their corresponding links:

In [ ]:
df_hrefs = df.loc[:, ["Movie Title", "link"]]

## Removing Duplicate Movies

Let's look at our extracted movie information and then remove duplicates. This will give us a clean list of unique movies that we need to collect additional information for.

In [ ]:
df_hrefs.head()

,Movie Title,link
0,Star Wars The Rise of Skywalker,/movie/Star-Wars-The-Rise-of-Skywalker-(2019)#...
1,Jumanji: The Next Level,/movie/Jumanji-The-Next-Level#tab=box-office
2,Little Women,/movie/Little-Women-(2019)#tab=box-office
3,Frozen II,/movie/Frozen-II-(2019)#tab=box-office
4,Spies in Disguise,/movie/Spies-in-Disguise-(2019)#tab=box-office


In [ ]:
df_hrefs = df_hrefs.drop_duplicates(subset=["link"])

## Generating Full URLs for Movie Details

For each movie, we need to visit two specific pages to collect all the information we want:
1. The summary page - containing general movie information
2. The cast and crew page - containing director information

We'll create a function to generate these URLs based on the link fragments we extracted earlier.

In [ ]:
base_url = "https://www.the-numbers.com"


# Define a function to generate the URLs from the href column
def create_urls(href):
    # Remove any anchor from the href string
    anchor_removed = re.sub(r"#.*$", "", href)
    # Create the full URLs based on the provided rule
    full_summary_url = f"{base_url}{anchor_removed}#tab=summary"
    full_cast_url = f"{base_url}{anchor_removed}#tab=cast-and-crew"
    return pd.Series([full_summary_url, full_cast_url])


df_hrefs[["full_summary_url", "full_cast_url"]] = df_hrefs["link"].apply(create_urls)


## Extracting Director Information

Next, we'll create a function to scrape the director's name from each movie's cast and crew page. This requires:

1. Navigating to the cast and crew page
2. Finding the production credits section
3. Locating the row that contains the director information
4. Extracting the director's name

This information will help us later analyze box office performance by director.

In [ ]:
def get_director(full_cast_url):
    cast_soup = extract_movies(full_cast_url)
    director = None
    prod_heading = cast_soup.find(
        "h1", string=re.compile(r"Production and Technical Credits", re.IGNORECASE)
    )
    if prod_heading:
        prod_table = prod_heading.find_next("table")
        if prod_table:
            for prow in prod_table.find_all("tr"):
                cells = prow.find_all("td")
                # Check that there are at least 3 cells and the third cell's text is "Director"
                if len(cells) >= 3 and cells[2].get_text(strip=True) == "Director":
                    director = cells[0].get_text(strip=True)
                    break
    return director


## Examining Our Movie Collection

Before we begin collecting additional data for each movie, let's look at the statistics of our movie dataset to understand the scope of our task.

In [ ]:
df_hrefs.describe()

,Movie Title,link,full_summary_url,full_cast_url
count,1447,1447,1447,1447
unique,1444,1447,1447,1447
top,Last Call,/movie/Star-Wars-The-Rise-of-Skywalker-(2019)#...,https://www.the-numbers.com/movie/Star-Wars-Th...,https://www.the-numbers.com/movie/Star-Wars-Th...
freq,2,1,1,1


## Collecting Director Information

Now we'll apply our get_director function to each movie in our collection. This process may take some time as we need to make a separate web request for each movie.

We use tqdm.pandas() to show a progress bar that will help us monitor the process.

In [ ]:
tqdm.pandas()
df_hrefs["director"] = df_hrefs["full_cast_url"].progress_apply(get_director)

100%|██████████| 1447/1447 [22:48<00:00,  1.06it/s]


## Checking the Results

Let's examine the results of our director data collection to ensure the process completed successfully and see how many directors we were able to identify.

In [ ]:
df_hrefs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1447 entries, 0 to 53918
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Movie Title       1447 non-null   object
 1   link              1447 non-null   object
 2   full_summary_url  1447 non-null   object
 3   full_cast_url     1447 non-null   object
 4   director          1430 non-null   object
dtypes: object(5)
memory usage: 67.8+ KB


## Creating a Function to Extract Movie Details

Now we'll define a function to extract additional information from each movie's summary page, including:

1. Running time
2. Domestic release date
3. International release date
4. Genre

This information will allow us to analyze box office performance by release timing, movie duration, and genre.

In [ ]:
def get_summary_info(url):
    summary_soup = extract_movies(url)

    running_time = None
    domestic_release = None
    international_release = None
    genre = None

    # Running Time
    rt_label = summary_soup.find("td", string=re.compile(r"Running Time:"))
    if rt_label:
        rt_value_td = rt_label.find_next_sibling("td")
        if rt_value_td:
            running_time = rt_value_td.get_text(strip=True)

    # Domestic Releases
    dom_label = summary_soup.find("td", string=re.compile(r"Domestic Releases:"))
    if dom_label:
        dom_value_td = dom_label.find_next_sibling("td")
        if dom_value_td:
            text_val = dom_value_td.get_text(" ", strip=True)
            # Extract a date like "January 1st, 2025"
            match = re.search(
                r"([A-Za-z]+\s+\d{1,2}(?:st|nd|rd|th)?,\s*\d{4})", text_val
            )
            if match:
                domestic_release = match.group(1)

    # International Releases
    intl_label = summary_soup.find("td", string=re.compile(r"International Releases:"))
    if intl_label:
        intl_value_td = intl_label.find_next_sibling("td")
        if intl_value_td:
            text_val = intl_value_td.get_text(" ", strip=True)
            match = re.search(
                r"([A-Za-z]+\s+\d{1,2}(?:st|nd|rd|th)?,\s*\d{4})", text_val
            )
            if match:
                international_release = match.group(1)

    # Genre
    genre_label = summary_soup.find("td", string=re.compile(r"Genre:"))
    if genre_label:
        genre_value_td = genre_label.find_next_sibling("td")
        if genre_value_td:
            genre_a = genre_value_td.find("a")
            if genre_a:
                genre = genre_a.get_text(strip=True)

    # Return as a Series so we can assign directly to multiple columns
    return pd.Series([running_time, domestic_release, international_release, genre])

## Collecting Additional Movie Information

Let's now apply our function to collect these additional details for each movie in our dataset. This will involve making another web request for each unique movie.

As before, we use a progress bar to track the process which may take some time to complete.

In [ ]:
df_hrefs[["running_time", "domestic_release", "international_release", "genre"]] = (
    df_hrefs["full_cast_url"].progress_apply(get_summary_info)
)


100%|██████████| 1447/1447 [21:17<00:00,  1.13it/s]


## Examining a Sample Movie Entry

Let's look at a single movie entry to see all the information we've collected. This helps us verify that our data collection was successful and gives us a better understanding of the data structure.

In [ ]:
df_hrefs.iloc[4, :]


,4
Movie Title,Spies in Disguise
link,/movie/Spies-in-Disguise-(2019)#tab=box-office
full_summary_url,https://www.the-numbers.com/movie/Spies-in-Dis...
full_cast_url,https://www.the-numbers.com/movie/Spies-in-Dis...
director,Nick Bruno
running_time,104 minutes
domestic_release,"December 25th, 2019"
international_release,"December 25th, 2019"
genre,Adventure


## Merging Daily Box Office Data with Movie Details

Now we'll combine our daily box office data with the detailed movie information we've collected. This merge operation will use the movie links as the common key between the two datasets.

The result will be a comprehensive dataset that includes both daily performance metrics and static movie characteristics.

In [ ]:
# Merge the two DataFrames on the 'link' column using a left join
merged_df = pd.merge(df, df_hrefs, on="link", how="left")

print(merged_df.head())

                     Movie Title_x       Distributor        Gross   %YD   %LW  \
0  Star Wars The Rise of Skywalker       Walt Disney  $17,076,472  +29%  -47%   
1          Jumanji: The Next Level     Sony Pictures  $10,909,345  +32%  -13%   
2                     Little Women     Sony Pictures   $5,242,720  +28%  -18%   
3                        Frozen II       Walt Disney   $5,029,104  +24%  +21%   
4                Spies in Disguise  20th Century Fox   $4,397,815  +20%   -8%   

  Theaters Per Theater   Total Gross Days In Release        Date  \
0    4,406      $3,876  $407,782,706              13  2020/01/01   
1    4,227      $2,581  $203,003,881              20  2020/01/01   
2    3,308      $1,585   $42,835,847               8  2020/01/01   
3    3,265      $1,540  $435,173,786              41  2020/01/01   
4    3,502      $1,256   $33,721,503               8  2020/01/01   

                                                link  \
0  /movie/Star-Wars-The-Rise-of-Skywalker-(2019)

## Saving the Complete Dataset

Let's save our merged dataset to a CSV file. This preserves all our collected data in its most complete form.

In [ ]:
merged_df.to_csv("final_data.csv", index=False)


## Cleaning the Final Dataset

Now that we've successfully merged our data, we can remove the URL columns that were necessary for scraping but aren't needed for analysis. This will make our final dataset cleaner and more focused on the analytical variables.

In [ ]:
# Drop the three columns that contain URL information
merged_df = merged_df.drop(columns=["link", "full_summary_url", "full_cast_url"])

print(merged_df.head())


                     Movie Title_x       Distributor        Gross   %YD   %LW  \
0  Star Wars The Rise of Skywalker       Walt Disney  $17,076,472  +29%  -47%   
1          Jumanji: The Next Level     Sony Pictures  $10,909,345  +32%  -13%   
2                     Little Women     Sony Pictures   $5,242,720  +28%  -18%   
3                        Frozen II       Walt Disney   $5,029,104  +24%  +21%   
4                Spies in Disguise  20th Century Fox   $4,397,815  +20%   -8%   

  Theaters Per Theater   Total Gross Days In Release        Date  \
0    4,406      $3,876  $407,782,706              13  2020/01/01   
1    4,227      $2,581  $203,003,881              20  2020/01/01   
2    3,308      $1,585   $42,835,847               8  2020/01/01   
3    3,265      $1,540  $435,173,786              41  2020/01/01   
4    3,502      $1,256   $33,721,503               8  2020/01/01   

                     Movie Title_y      director running_time  \
0  Star Wars The Rise of Skywalker   J.

## Saving the Clean Final Dataset

Finally, we'll save our clean, analysis-ready dataset to a CSV file. This dataset represents the culmination of our web scraping efforts and contains comprehensive box office data for movies from 2020 to early 2025.

This dataset is now ready for various analyses, such as:
- Trends in box office performance over time
- Comparisons between different genres
- Effects of the pandemic on theater attendance
- Patterns in movie release timing
- Performance differences across distributors

In [ ]:
merged_df.to_csv("theNumbersDataset.csv", index=False)


# Conclusion

In this notebook, we've successfully:

1. Built robust web scraping tools specifically designed for The-Numbers.com
2. Collected daily box office data spanning from 2020 to early 2025
3. Extracted detailed information about each unique movie
4. Combined everything into a comprehensive dataset for analysis

